In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Post-Traumatic_Stress_Disorder"
cohort = "GSE81761"

# Input paths
in_trait_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder"
in_cohort_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder/GSE81761"

# Output paths
out_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/GSE81761.csv"
out_gene_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE81761.csv"
out_clinical_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/clinical_data/GSE81761.csv"
json_path = "../../output/preprocess/Post-Traumatic_Stress_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene Expression Pathways Implicated in Posttraumatic Stress Disorder and Symptomatic Improvement"
!Series_summary	"Military service members often return from deployment with a multiplicity of injuries, including mild traumatic brain injury, depression, and sleep disorders, which obsures diagnosis of PTSD symptoms and complicates treatment of PTSD. In order to understand the biological mechanisms underlying PTSD, gene expression profiles of military service members with and without PTSD were compared. Additionally, gene expression was examined based on intrusion symptoms, a distinct subtype of PTSD symptoms, and on improvement of PTSD symptoms at a three month follow up. RNA was extracted from blood samples and hybridized to the HG-U133_Plus_2 Affymetrix chip."
!Series_overall_design	"Gene expression of subjects with PTSD (n=39) were compared to controls without PTSD (n=27) at baseline. Further analysis of gene expression for subjects with PTSD at 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# The background information indicates gene expression data is available:
# "RNA was extracted from blood samples and hybridized to the HG-U133_Plus_2 Affymetrix chip."
# HG-U133_Plus_2 is a standard microarray for gene expression
is_gene_available = True

# 2.1 Data Availability
# Review the sample characteristics dictionary to identify relevant rows
# For PTSD (trait): Row 1 contains 'case/control: PTSD' and 'case/control: No PTSD'
trait_row = 1

# For Age: Row 5 contains age information
age_row = 5

# For Gender: Row 4 contains 'Sex: Male' and 'Sex: Female'
gender_row = 4

# 2.2 Data Type Conversion

def convert_trait(value: str) -> int:
    """Convert PTSD trait value to binary format (1 for PTSD, 0 for No PTSD)."""
    if not value or ":" not in value:
        return None
    
    value = value.split(":", 1)[1].strip()
    if "PTSD" in value and "No" not in value:
        return 1
    elif "No PTSD" in value:
        return 0
    return None

def convert_age(value: str) -> float:
    """Convert age value to continuous format."""
    if not value or ":" not in value:
        return None
    
    value = value.split(":", 1)[1].strip()
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> int:
    """Convert gender value to binary format (0 for Female, 1 for Male)."""
    if not value or ":" not in value:
        return None
    
    value = value.split(":", 1)[1].strip().lower()
    if "male" in value and "fe" not in value:
        return 1
    elif "female" in value:
        return 0
    return None

# 3. Save Metadata - Initial validation
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Load the clinical data
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of extracted clinical features:")
        print(preview)
        
        # Save the clinical data to the output file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print(f"Clinical data file not found at {clinical_data_path}")


Clinical data file not found at ../../input/GEO/Post-Traumatic_Stress_Disorder/GSE81761/clinical_data.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene data dimensions: 54675 genes × 109 samples


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers like '1007_s_at', '1053_at', these appear to be Affymetrix probe IDs
# from an Affymetrix microarray platform, not standard human gene symbols.
# These probe IDs need to be mapped to gene symbols for proper biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene identifiers and decide which columns to use
# From the annotation preview, we can see:
# - 'ID' column contains probe IDs like '1007_s_at' which matches gene_data.index
# - 'Gene Symbol' column contains the gene symbols we need

# 2. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')

# Preview the mapping to confirm
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print statistics about the gene data after mapping
print(f"\nGene data after mapping dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


Gene mapping preview (first 5 rows):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A

Gene data after mapping dimensions: 21278 genes × 109 samples
First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Re-extract the gene expression data to ensure we have it properly defined
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)

# Extract gene annotation data for mapping
gene_annotation = get_gene_annotation(soft_file)

# Based on previous output in Step 5, we can see the column is 'Gene Symbol'
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Apply gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 normalized gene symbols: {normalized_gene_data.index[:5].tolist() if len(normalized_gene_data) > 0 else 'No genes after normalization'}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Re-extract clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# From Step 2, we identified the correct rows
trait_row = 1  # 'case/control: PTSD', 'case/control: No PTSD'
age_row = 5    # 'age: 30', 'age: 38', etc.
gender_row = 4  # 'Sex: Male', 'Sex: Female'

def convert_trait(value):
    """Convert PTSD information to binary (0: No PTSD, 1: PTSD)"""
    if value is None or pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if "PTSD" in value and "No" not in value:
        return 1
    elif "No PTSD" in value:
        return 0
    return None

def convert_age(value):
    """Convert age information to numeric (continuous) values"""
    if value is None or pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender information to binary (0: female, 1: male)"""
    if value is None or pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    if "female" in value:
        return 0
    elif "male" in value and "fe" not in value:
        return 1
    return None

# Extract clinical features with correct row indices and conversion functions
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Debug: Show preview of clinical data
print("Preview of clinical data:")
print(preview_df(selected_clinical_df))

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait is biased
print("\nChecking for bias in the trait variable:")
# The trait in this dataset is binary (PTSD vs No PTSD)
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation
is_trait_available = True  # We confirmed trait data is available in Step 2
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset studies blood gene expression profiles in PTSD patients compared to controls, with assessment of symptom improvement at follow-up."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: (19845, 109)
First 5 normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1']


Normalized gene data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE81761.csv


Preview of clinical data:
{'GSM2175165': [1.0, 30.0, 1.0], 'GSM2175166': [0.0, 38.0, 1.0], 'GSM2175167': [1.0, 39.0, 1.0], 'GSM2175168': [1.0, 38.0, 1.0], 'GSM2175169': [1.0, 23.0, 1.0], 'GSM2175170': [0.0, 48.0, 1.0], 'GSM2175171': [0.0, 49.0, 1.0], 'GSM2175172': [0.0, 34.0, 1.0], 'GSM2175173': [1.0, 33.0, 1.0], 'GSM2175174': [1.0, 45.0, 1.0], 'GSM2175175': [1.0, 25.0, 1.0], 'GSM2175176': [0.0, 25.0, 1.0], 'GSM2175177': [1.0, 30.0, 1.0], 'GSM2175178': [1.0, 39.0, 1.0], 'GSM2175179': [1.0, 23.0, 1.0], 'GSM2175180': [1.0, 22.0, 1.0], 'GSM2175181': [1.0, 46.0, 1.0], 'GSM2175182': [0.0, 35.0, 1.0], 'GSM2175183': [0.0, 22.0, 1.0], 'GSM2175184': [1.0, 23.0, 1.0], 'GSM2175185': [0.0, 48.0, 1.0], 'GSM2175186': [1.0, 23.0, 1.0], 'GSM2175187': [0.0, 49.0, 1.0], 'GSM2175188': [1.0, 38.0, 1.0], 'GSM2175189': [1.0, 25.0, 1.0], 'GSM2175190': [1.0, 33.0, 1.0], 'GSM2175191': [1.0, 30.0, 1.0], 'GSM2175192': [0.0, 36.0, 1.0], 'GSM2175193': [0.0, 43.0, 1.0], 'GSM2175194': [0.0, 34.0, 1.0], 'GSM2175195':

Data shape after handling missing values: (109, 19848)

Checking for bias in the trait variable:
For the feature 'Post-Traumatic_Stress_Disorder', the least common label is '0.0' with 47 occurrences. This represents 43.12% of the dataset.
The distribution of the feature 'Post-Traumatic_Stress_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 26.0
  50% (Median): 33.0
  75%: 39.0
Min: 21.0
Max: 52.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 5 occurrences. This represents 4.59% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/GSE81761.csv
